In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import os
import glob

from scipy.stats import linregress

In [ ]:
clean_list = glob.glob(r'C:\Users\BOIL_PO\Desktop\分類部分\VV\基於統計\vv_clean\*csv')
print(len(clean_list))
clean_list[0]

# Time sheet

In [ ]:
dfss = pd.read_excel(r'C:\Users\BOIL_PO\Desktop\分類部分\VV\基於統計\Time_sheet.xlsx', sheet_name='Time_VV')
dfss

## 製作特徵萃取函數

In [ ]:
df = pd.read_csv(clean_list[0], index_col= 'time(s)')  
df

In [ ]:
# 取出人對應的時間

people_id = int(clean_list[0].split("\\")[-1].split("_")[0].split("-")[1])
date = int(clean_list[0].split("\\")[-1].split("_")[0].split("-")[2])
print(people_id, date)

data_get = dfss[(dfss['Chart No']==people_id)  &	(dfss['Date'] == date)]
data_get


In [ ]:
time_dura  = data_get[['Baseline', 'Stage1', 'Stage2',	'Stage3', 'Stage4',	'Stage5']].cumsum(axis= 1)*60
time_series = [0] + time_dura.values[0].tolist()
time_series

In [ ]:
time_label = [0, 1, 2, 3, 4, 5]

In [ ]:
df['stage'] = pd.cut(df.index, bins=time_series, labels=time_label, include_lowest=True)
df_data = df.dropna(subset=['stage'])
df_data

## 只要HbO2

# <font color = yellow>宇翰是三channel合併為一</font>

In [ ]:
dfs = pd.DataFrame()
df_drop = df_data.drop(columns=['TSI%'])

for i in range(len(df_drop.columns)//3):
    temp = df_drop[[df_drop.columns[i], df_drop.columns[i+3], df_drop.columns[i+6]]].mean(axis= 1).values
    dfs[df_drop.columns[i].split("_")[-1]] = temp
 
dfs['stage'] = df_data['stage'].values
dfs.index = df_data.index
dfs

In [ ]:
# stage mean
def stage_begin_slope(x):
    xxx = x.loc[(x.index[0] <= x.index) & (x.index <  x.index[0]+8)]
    return linregress(xxx.index, xxx).slope




def stage_mean_rela(df):

    df_len = int(len(df.columns)) - 1


    if not "stage" in df.columns:
        print('No stage column')
        return 0

    
    s_mean = df.groupby('stage').mean()
    smean = s_mean.rename(columns= lambda x: x + '_mean')

    s_activate = s_mean - s_mean.iloc[0]
    sactivate = s_activate.rename(columns= lambda x: x + '_activate')

    s_stage_diff = s_mean.diff()
    sstage_diff = s_stage_diff.rename(columns= lambda x: x + '_mean_diff')

    s_slope = df.groupby('stage').agg(stage_begin_slope)

    sslope = s_slope.rename(columns= lambda x: x + '_begin_slope')

    dfd = pd.concat([smean, sactivate, sstage_diff, sslope], axis= 1)
        
    return dfd

In [ ]:
xx = stage_mean_rela(dfs)
xx

In [ ]:
df_save = pd.DataFrame()

for mm in xx.stack().index:

    if mm in [(0, 'HbO2_activate'), (0, 'tHb_activate'), (0, 'HHb_activate'), (1, 'HbO2_activate'), (1, 'HHb_activate'), (1, 'tHb_activate')]:
        continue
    i, row = mm
    df_save[row+ "_stage" + str(i)] =  [xx.stack()[mm]]

In [ ]:
df_save

# 批次行為

In [ ]:
df_all = pd.DataFrame()
time_label = [0, 1, 2, 3, 4, 5]
for csv_data in clean_list:
    df = pd.read_csv(csv_data, index_col= 'time(s)')  

    people_id = int(csv_data.split("\\")[-1].split("_")[0].split("-")[1])
    date = int(csv_data.split("\\")[-1].split("_")[0].split("-")[2])

    data_get = dfss[(dfss['Chart No']==people_id) & (dfss['Date'] == date)]

    sofa = data_get['SOFA score'].values

    # time dura
    time_dura  = data_get[['Baseline', 'Stage1', 'Stage2',	'Stage3', 'Stage4',	'Stage5']].cumsum(axis= 1)*60
    time_series = [0] + time_dura.values[0].tolist()

    df['stage'] = pd.cut(df.index, bins=time_series, labels=time_label, include_lowest=True)
    df_data = df.dropna(subset=['stage'])

    dfs = pd.DataFrame()
    df_drop = df_data.drop(columns=['TSI%'])

    for i in range(len(df_drop.columns)//3):
        temp = df_drop[[df_drop.columns[i], df_drop.columns[i+3], df_drop.columns[i+6]]].mean(axis= 1).values
        dfs[df_drop.columns[i].split("_")[-1]] = temp
    
    dfs['stage'] = df_data['stage'].values
    dfs.index = df_data.index


    xx = stage_mean_rela(dfs)

    df_save = pd.DataFrame()

    for mm in xx.stack().index:

        if mm in [(0, 'HbO2_activate'), (0, 'tHb_activate'), (0, 'HHb_activate'), (1, 'HbO2_activate'), (1, 'HHb_activate'), (1, 'tHb_activate')]:
            continue
        i, row = mm
        df_save[row+ "_stage" + str(i)] =  [xx.stack()[mm]]

    df_save['SOFA score'] = sofa

    df_save.index = [str(people_id) + "_" + str(date)]
    
    df_all = pd.concat([df_all, df_save])
    
df_all

In [ ]:
df_all.columns = [x.replace("stage0", "baseline") for x in df_all.columns]
df_all

In [ ]:
df_all.to_csv("Feature_Extracition.csv", index= True)